In [1]:
pwd

'/Users/gemunu/kaggle'

In [2]:
import pandas as pd
import seaborn as sns
#get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import numpy as np
import xgboost as xgb
from  pandas.tseries.common import DatetimeProperties as dt

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
import warnings

In [4]:
warnings.filterwarnings('ignore')

In [5]:
train=pd.read_csv('train.csv')

In [6]:
import ipython_memory_usage.ipython_memory_usage as imu

In [7]:
imu.start_watching_memory()

In [7] used -0.3164 MiB RAM in 16.91s, peaked 0.00 MiB above current, total RAM usage 1035.49 MiB


In [8]:
test=pd.read_csv('test.csv')

In [8] used -93.5625 MiB RAM in 26.69s, peaked 262.63 MiB above current, total RAM usage 941.93 MiB


In [12]:
test.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_device_type,first_browser,month_created,day_created,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35,facebook,0,en,direct,...,iPhone,Mobile Safari,7,1,5uwns89zht,show,view,user_profile,-unknown-,79
1,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35,facebook,0,en,direct,...,iPhone,Mobile Safari,7,1,5uwns89zht,search,click,view_search_results,-unknown-,17962
2,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35,facebook,0,en,direct,...,iPhone,Mobile Safari,7,1,5uwns89zht,search,click,view_search_results,-unknown-,64883
3,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35,facebook,0,en,direct,...,iPhone,Mobile Safari,7,1,5uwns89zht,show,view,p3,-unknown-,31180
4,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35,facebook,0,en,direct,...,iPhone,Mobile Safari,7,1,5uwns89zht,authenticate,submit,login,iPhone,NaN


In [12] used -176.0391 MiB RAM in 0.20s, peaked 19.26 MiB above current, total RAM usage 1415.96 MiB


In [11]:
train.drop(train.columns[0],axis=1,inplace=True)
test.drop(test.columns[0],axis=1,inplace=True)

In [11] used 643.6289 MiB RAM in 7.38s, peaked 112.81 MiB above current, total RAM usage 1592.00 MiB


In [13]:
target=train.pop('country_destination')
id_test=test['id']
train.drop('timestamp_first_active',axis=1,inplace=True)
test.drop('timestamp_first_active',axis=1,inplace=True)
train.drop('date_first_booking',axis=1,inplace=True)
test.drop('date_first_booking',axis=1,inplace=True)
train.drop(['id','user_id'],axis=1,inplace=True)
test.drop(['id','user_id'],axis=1,inplace=True)

In [13] used -431.0938 MiB RAM in 21.55s, peaked 396.72 MiB above current, total RAM usage 984.87 MiB


In [ ]:
train['month_created']=train['date_account_created'].apply(pd.to_datetime).dt.month
train['day_created']=train['date_account_created'].apply(pd.to_datetime).dt.dayofweek
test['month_created']=test['date_account_created'].apply(pd.to_datetime).dt.month
test['day_created']=test['date_account_created'].apply(pd.to_datetime).dt.dayofweek

In [ ]:
test=pd.merge(test,sessions,left_on='id',right_on='user_id',how='left')

In [ ]:
train=pd.merge(train,sessions,left_on='id',right_on='user_id',how='left')

In [15]:
len(test)

4995712

In [15] used 0.0703 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 990.34 MiB


In [16]:
train.drop('date_account_created',axis=1,inplace=True)
test.drop('date_account_created',axis=1,inplace=True)

In [16] used -427.0078 MiB RAM in 6.19s, peaked 524.38 MiB above current, total RAM usage 563.33 MiB


In [17]:
cat_vars=['first_affiliate_tracked','gender', 'signup_method', 'language',
       'affiliate_channel', 'affiliate_provider',
       'signup_app', 'first_device_type', 'first_browser','action','action_type','action_detail','device_type']


In [17] used 4.6250 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 567.96 MiB


In [18]:
train['first_affiliate_tracked'].fillna('missed',inplace=True)
test['first_affiliate_tracked'].fillna('missed',inplace=True)
train['action_detail'].fillna('missed',inplace=True)
test['action_detail'].fillna('missed',inplace=True)
train['action'].fillna('missed',inplace=True)
test['action'].fillna('missed',inplace=True)
train['action_type'].fillna('missed',inplace=True)
test['action_type'].fillna('missed',inplace=True)
train['device_type'].fillna('missed',inplace=True)
test['device_type'].fillna('missed',inplace=True)

In [18] used 348.9570 MiB RAM in 3.05s, peaked 0.00 MiB above current, total RAM usage 916.91 MiB


In [19]:
le=LabelEncoder()
for i in cat_vars:
    train[i]=le.fit_transform(train[i])
    test[i]=le.fit_transform(test[i])

In [19] used 468.3477 MiB RAM in 157.94s, peaked 0.00 MiB above current, total RAM usage 1385.26 MiB


In [22]:
y=le.fit_transform(target)

In [22] used -25.5547 MiB RAM in 5.63s, peaked 13.42 MiB above current, total RAM usage 1049.38 MiB


In [23]:
test.fillna(-1,inplace=True)
train.fillna(-1,inplace=True)

In [23] used 135.8242 MiB RAM in 7.00s, peaked 673.52 MiB above current, total RAM usage 1185.20 MiB


In [24]:
dtest=xgb.DMatrix(test,missing=-1)

In [24] used -388.1094 MiB RAM in 7.71s, peaked 1307.38 MiB above current, total RAM usage 797.09 MiB


In [25]:
dtrain=xgb.DMatrix(train,label=y,missing=-1)

In [25] used 182.9805 MiB RAM in 12.57s, peaked 859.39 MiB above current, total RAM usage 980.07 MiB


In [123]:
param = {'bst:max_depth':6, 'bst:eta':0.3, 'silent':1, 'objective':'multi:softprob','num_class':12}
param["colsample_bytree"] = 0.30
param["subsample"] = 0.60
param["min_child_weight"] = 1
plst=list(param.items())

In [123] used 0.0586 MiB RAM in 0.14s, peaked 0.00 MiB above current, total RAM usage 35.11 MiB


In [30]:
bst=xgb.train(plst,dtrain,num_boost_round=10)

In [30] used 40.8477 MiB RAM in 645.04s, peaked 416.11 MiB above current, total RAM usage 1042.02 MiB


In [31]:
bst_pred=bst.predict(dtest)

In [31] used -375.7031 MiB RAM in 47.14s, peaked 0.00 MiB above current, total RAM usage 666.31 MiB


In [113]:
%%time
xgbc=xgb.sklearn.XGBClassifier(max_depth=3,n_estimators=2,learning_rate=0.1,objective='multi:softprob',missing=-1)

CPU times: user 184 µs, sys: 1.66 ms, total: 1.84 ms
Wall time: 4.55 ms
In [113] used -1178.7188 MiB RAM in 0.23s, peaked 0.00 MiB above current, total RAM usage 69.81 MiB


In [134]:
%%time
rfc=RandomForestClassifier(n_estimators=60,max_depth=3,criterion='gini',max_features=0.33)
rfc.fit(train,y)

CPU times: user 6min 23s, sys: 1min 8s, total: 7min 32s
Wall time: 7min 22s
In [134] used 314.0039 MiB RAM in 442.98s, peaked 670.79 MiB above current, total RAM usage 1061.22 MiB


In [ ]:
rfcp=rfc.predict_proba(test)

In [32]:
v1,v2,v3,v4,v5=[],[],[],[],[]
for i in range(len(bst_pred)):
    v1.append(np.argsort(bst_pred[i])[::-1][:5][0])
    v2.append(np.argsort(bst_pred[i])[::-1][:5][1])
    v3.append(np.argsort(bst_pred[i])[::-1][:5][2])
    v4.append(np.argsort(bst_pred[i])[::-1][:5][3])
    v5.append(np.argsort(bst_pred[i])[::-1][:5][4])

In [32] used -203.4219 MiB RAM in 65.58s, peaked 2.01 MiB above current, total RAM usage 462.89 MiB


In [33]:
result_df=pd.DataFrame({'v1':v1,'v2':v2,'v3':v3,'v4':v4,'v5':v5})

In [33] used 535.6836 MiB RAM in 9.21s, peaked 0.00 MiB above current, total RAM usage 998.57 MiB


In [34]:
result_df['id']=id_test

In [34] used -296.1641 MiB RAM in 0.74s, peaked 70.75 MiB above current, total RAM usage 702.41 MiB


In [35]:
res_group=result_df.groupby(by='id')

In [35] used -73.5391 MiB RAM in 0.16s, peaked 0.00 MiB above current, total RAM usage 628.87 MiB


In [46]:
vf=[]
for i in range(10):
    vf.extend((int(res_group['v1'].mean()[i]),int(res_group['v2'].mean()[i]),int(res_group['v3'].mean()[i]),int(res_group['v4'].mean()[i]),int(res_group['v5'].mean()[i])))

In [46] used -0.8633 MiB RAM in 3.11s, peaked 0.00 MiB above current, total RAM usage 312.87 MiB


In [47]:
vf=[]
for i in range(10):
    vf.append((int(res_group['v1'].mean()[i]),int(res_group['v2'].mean()[i]),int(res_group['v3'].mean()[i]),int(res_group['v4'].mean()[i]),int(res_group['v5'].mean()[i])))

In [47] used 0.0469 MiB RAM in 3.06s, peaked 0.00 MiB above current, total RAM usage 312.91 MiB


In [59]:
kk=res_group.mean()

In [59] used -19.0898 MiB RAM in 0.69s, peaked 185.21 MiB above current, total RAM usage 286.02 MiB


In [89]:
l=kk.values.astype(int).ravel()

In [89] used 2.3789 MiB RAM in 0.13s, peaked 0.00 MiB above current, total RAM usage 178.52 MiB


In [98]:
result_df

,v1,v2,v3,v4,v5,id
0,7,10,11,4,6,5uwns89zht
1,7,10,11,4,6,5uwns89zht
2,7,10,11,4,6,5uwns89zht
3,7,10,11,4,6,5uwns89zht
4,7,10,11,4,6,5uwns89zht
5,7,10,11,4,6,5uwns89zht
6,7,10,11,6,4,5uwns89zht
7,7,10,11,4,6,5uwns89zht
8,7,10,11,4,6,jtl0dijy2j
9,7,10,11,4,6,jtl0dijy2j


In [98] used -194.2695 MiB RAM in 0.19s, peaked 0.00 MiB above current, total RAM usage 30.64 MiB


In [82]:
ids=result_df.id.unique().tolist()

In [82] used 85.0742 MiB RAM in 0.48s, peaked 0.00 MiB above current, total RAM usage 175.83 MiB


In [93]:
idss,cts=[],[]
for i in range(len(ids)):
    idx = ids[i]
    idss += [idx] * 5

cts = le.inverse_transform(l)

In [93] used -17.3477 MiB RAM in 0.21s, peaked 0.00 MiB above current, total RAM usage 161.23 MiB


In [96]:
cts

array(['NDF', 'US', 'other', ..., 'other', 'GB', 'FR'], dtype=object)

In [96] used 0.0312 MiB RAM in 0.13s, peaked 0.00 MiB above current, total RAM usage 162.12 MiB


In [97]:
sub = pd.DataFrame(np.column_stack((idss, cts)), columns=['id', 'country'])
sub.to_csv('sub.csv',index=False)
pd.read_csv('sub.csv')

,id,country
0,5uwns89zht,NDF
1,5uwns89zht,US
2,5uwns89zht,other
3,5uwns89zht,FR
4,5uwns89zht,IT
5,jtl0dijy2j,NDF
6,jtl0dijy2j,US
7,jtl0dijy2j,other
8,jtl0dijy2j,FR
9,jtl0dijy2j,AU


In [97] used 62.7930 MiB RAM in 0.85s, peaked 4.59 MiB above current, total RAM usage 224.91 MiB
